# 의사결정 트리

- 특징과 목푯값 간 관계에서 트리 기반 규칙을 찾아내는 지도학습 모형
- 회귀 및 분류 작업 모두에 활용 가능

In [1]:
# 학습에 사용할 데이터
feature_names = ['carat', 'price', 'cut']
feature_samples = [[0.21, 327, 'Okay'],
                    [0.39, 897, 'Perfect'],
                    [0.50, 1122, 'Perfect'],
                    [0.76, 907, 'Okay'],
                    [0.87, 2757, 'Okay'],
                    [0.98, 2865, 'Okay'],
                    [1.13, 3045, 'Perfect'],
                    [1.34, 3914, 'Perfect'],
                    [1.67, 4849, 'Perfect'],
                    [1.81, 5688, 'Perfect']]

### 클래스 설계
- `DecisionNode`: 결정 노드 (질문과 두개의 가지로 구성)
- `LeafNode`: 리프 노드 (분류 완료된 데이터 개수)

In [2]:
class DecisionNode:
    def __init__(self, question):
        self.question = question
        self.branch_true = None
        self.branch_false = None

class LeafNode:
    def __init__(self, samples):
        self.predictions = count_class(samples)

## 주요 함수 설계
- `generate_possible_questions()`: 가능한 질문 생성
- `split_data()`: 질문에 근거해 TRUE/FALSE 데이터 분리
- `count_class()`: 각 클래스 별 데이터 개수 맵핑
- `calculate_impurity()`: 불순도 계산(Gini 계수 혹은 Entropy)
- `calculate_information_gain()`: 정보 이득 계산

In [3]:
# 가능한 질문 모두 생성
# 질문형태를 잘 고려해야함 (현재는 2가지 정보가 담김)
def generate_possible_questions(samples):
    questions = set()
    num_features = len(samples[0]) - 1
    for feature_index in range(num_features):
        values = set(sample[feature_index] for sample in samples)
        for value in values:
            questions.add((feature_index, value))
    return questions

In [4]:
# 질문에 근거해 데이터 나누기(true, false)
def split_data(samples, question):
    true_data, false_data = [], []
    feature_index, value = question

    for sample in samples:
        if sample[feature_index] >= value:
            true_data.append(sample)
        else:
            false_data.append(sample)
    return true_data, false_data

In [5]:
# 클래스 그룹화 맵: 데이터 샘플 리스트에서 각 클래스(레이블)의 개수를 세어 딕셔너리 형태로 반환
def count_class(samples):
    counts = {}
    for sample in samples:
        label = sample[-1]
        if label  not in counts:
            counts[label] = 0
        counts[label] += 1 
    return counts

In [6]:
# Gini 계산
def calculate_impurity(samples):
    counts = count_class(samples)
    total_samples = len(samples)
    impurity = 1
    for label in counts:
        prob = counts[label] / total_samples
        impurity -= prob**2
    return impurity


In [13]:
# 정보이득 계산
def calculate_information_gain(impurity, true_samples, false_samples):
    total_samples = len(true_samples) + len(false_samples) 
    weighted_avg_impurity = (
        (len(true_samples) / total_samples) * calculate_impurity(true_samples) +
        (len(false_samples) / total_samples) * calculate_impurity(false_samples)
    )
    return impurity - weighted_avg_impurity

### 의사결정 트리 학습 알고리즘
- `decision_tree()`: 학습 데이터로부터 규칙 학습
  1. 초기 데이터 불순도 계산
  2. 질문 생성
  3. 데이터 분리 후 불순도 계산
  4. 정보이득 계산
  5. 최고 정보이득 가진 질문으로 의사결정 노드 추가

In [26]:
# 의사결정 트리 학습
def decision_tree(samples):
    impurity = calculate_impurity(samples)
    if impurity == 0:
        return LeafNode(samples)
    questions = generate_possible_questions(samples)
    best_gain = 0
    best_question = None
    best_splits = None

    for question in questions:
        true_samples, false_samples = split_data(samples, question)
        if not true_samples or not false_samples:
            continue
        gain = calculate_information_gain(impurity, true_samples, false_samples)
        if gain > best_gain:
            best_gain = gain
            best_question = question
            best_splits = (true_samples, false_samples)
    decision_node = DecisionNode(best_question)
    decision_node.branch_true = decision_tree(best_splits[0])
    decision_node.branch_false = decision_tree(best_splits[1])

    return decision_node

##### 트리 출력 함수

In [27]:
# 학습된 트리를를 출력 (재귀적으로 모든 노드를 탐색)
def print_tree(node):
    if isinstance(node, LeafNode): # node가 LeafNode의 인스턴스인지 확인하는 파이썬 내장 함수
        print("[Leaf]", node.predictions)
        return

    print(f"[Decision] >>'{feature_names[node.question[0]]} >= {node.question[1]}?'")
    print('--> True:')
    print_tree(node.branch_true)
    print('--> False:')
    print_tree(node.branch_false)

In [28]:
tree = decision_tree(feature_samples) # 트리 학습
print_tree(tree) # 학습된 트리 출력

[Decision] >>'price >= 3045?'
--> True:
[Leaf] {'Perfect': 4}
--> False:
[Decision] >>'carat >= 0.76?'
--> True:
[Leaf] {'Okay': 3}
--> False:
[Decision] >>'carat >= 0.39?'
--> True:
[Leaf] {'Perfect': 2}
--> False:
[Leaf] {'Okay': 1}


#### 테스트 데이터

In [29]:
test_samples = [
    [0.25, 500, 'Okay'],
    [0.42, 1000, 'Perfect'],
    [0.75, 850, 'Okay'],
    [1.00, 2000, 'Okay'],
    [1.20, 3200, 'Perfect'],
    [1.50, 4000, 'Perfect'],
    [1.80, 5000, 'Perfect'],
    [1.90, 5700, 'Perfect'],  
    [0.80, 1050, 'Okay'],
    [0.30, 700, 'Okay']        
]

#### 학습된 트리 활용 예측

In [30]:
def predict(tree, sample):
    if isinstance(tree, LeafNode):
        return next(iter(tree.predictions))

    feature_index, value = tree.question
    if sample[feature_index] >= value:
        return predict(tree.branch_true, sample)
    else:
        return predict(tree.branch_false, sample)

# 테스트 데이터로 예측
y_test = [sample[-1] for sample in test_samples]  # 실제 라벨
y_pred = [predict(tree, sample) for sample in test_samples]  # 모델 예측

In [ ]:
#같은지 다른지 비교 
comparison = [y_t == y_p for y_t, y_p in zip(y_test, y_pred)]
f"Accuracy: {sum(comparison)*10}%"

'Accuracy: 90%'

### Scikit-Learn 활용 의사결정 트리

- 더 많은 데이터 활용하기 위해 diamonds.csv 파일 사용

In [ ]:
import pandas as pd
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics

- 아래 데이터 읽기 및 전처리 파트에는 pandas 라는 데이터 처리를 위한 라이브러리가 사용되고 있음 (머신러닝에서 가장 많이 사용되는 라이브러리 중 하나로 데이터 전처리에 필요한 여러 기능을 제공함)

In [5]:
# 데이터 읽기 (pandas 활용)
df = pd.read_csv('diamonds.csv')

# 데이터 전처리
# 1. 불필요한 특징 제거
df = df.drop(columns=['no', 'depth', 'table', 'x size', 'y size', 'z size'])

# 2. 카테고리 인코딩(머신러닝 모델이 읽을 수 없는 문자형 범주를 수치로 변경하기 위한 인코딩 값 매칭)
encoding_categories = {
    'cut': {'Fair': 1, 'Good': 1, 'Very Good': 2, 'Premium': 2, 'Ideal': 2},
    'color': {'J': 1, 'I': 2, 'H': 3, 'G': 4, 'F': 5, 'E': 6, 'D': 7},
    'clarity': {'I1': 1, 'SI2': 2, 'SI1': 3, 'VS2': 4, 'VS1': 5, 'VVS2': 6, 'VVS1': 7, 'IF': 8}
}

df['cut'] = df['cut'].map(encoding_categories['cut'])
df['color'] = df['color'].map(encoding_categories['color'])
df['clarity'] = df['clarity'].map(encoding_categories['clarity'])

# 3. 특징(X)과 라벨(Y) 분리
X = df.drop(columns=['cut'])  # 특징
y = df['cut']  # 라벨

In [4]:
# 데이터셋의 전체를 학습, 평가 데이터(8:2)로 나눔
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
# 결정 트리 모델 학습


DecisionTreeClassifier()

In [ ]:
# 예측


In [ ]:
# 평가: 혼동행렬 및 정확도
confusion_matrix = metrics.confusion_matrix(y_test, predictions)
print("Confusion Matrix:\n", confusion_matrix)

accuracy = metrics.accuracy_score(y_test, predictions)
print("Prediction Accuracy:\n", accuracy)

Confusion Matrix:
 [[ 442  880]
 [ 916 8550]]
Prediction Accuracy:
 0.8335187245087133
